In [1]:
from keras.models import load_model

import matplotlib.pylab as plt
import pandas as pd
import numpy as np
import time
from IPython import display
%matplotlib inline

import rospy
import tensorflow as tf

from sensor_msgs.msg import JointState
from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import HoverTool
from bokeh.plotting import figure 

Using TensorFlow backend.


Couldn't import dot_parser, loading of dot files will not be possible.


In [ ]:
grasp_model = load_model("./model.h5")
graph = tf.get_default_graph()
global graph

In [ ]:
joints = ['H1_F1J2', 'H1_F1J3', 'H1_F1J1', 'H1_F3J1', 'H1_F3J2', 'H1_F3J3', 'H1_F2J1', 'H1_F2J3', 'H1_F2J2']

nb_points = 100
qualities = [0]*nb_points
qualities_id = range(nb_points)

output_notebook()

p = figure(title="simple line example", plot_height=300, plot_width=600, y_range=(0,1))
r = p.line(qualities_id, qualities, color="#2222aa", line_width=3)
global p
global r

show(p, notebook_handle=True)

def joint_states_cb(msg):
    input_vector = []
    for joint in joints:
        id_joint = msg.name.index(joint)
        input_vector.append(msg.velocity[id_joint])
        input_vector.append(msg.effort[id_joint])
    with graph.as_default():
        qualities.pop(0)
        qual = grasp_model.predict(np.array([input_vector]))
        qualities.append(qual)
        rospy.logdebug( "new quality: " + str(qual))
    
    r.data_source.data['y'] = qualities
    push_notebook()
    

rospy.init_node("grasp_quality", anonymous=True)
sub = rospy.Subscriber("/joint_states", JointState, joint_states_cb, queue_size=1)


rospy.spin()

Loading BokehJS ...